# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)
#print(g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_wx_df = pd.read_csv("../output_data/cities_weather.csv")
city_wx_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
0,Chokurdakh,RU,2126123,147.9167,70.6333,-34.26,-34.26,83,34,7.45
1,Abu Dhabi,AE,292968,54.3667,24.4667,70.83,71.60,78,0,5.75
2,Vaini,TO,4032243,-175.2000,-21.2000,82.40,82.40,69,20,12.66
3,Forsytheganj,BY,10722858,29.7333,52.6333,33.48,33.48,97,100,2.21
4,Ushuaia,AR,3833367,-68.3000,-54.8000,42.80,42.80,87,75,31.07
...,...,...,...,...,...,...,...,...,...,...
547,San Rafael,AR,3836669,-68.3301,-34.6177,73.09,73.09,29,0,5.64
548,Ifanadiana,MG,1064366,47.6333,-21.3000,69.57,69.57,93,27,1.16
549,Santa Cruz de la Sierra,BO,3904906,-63.1667,-17.8000,79.92,80.60,83,75,8.05
550,Vestmannaeyjar,IS,3412093,-20.2734,63.4427,38.70,38.70,80,70,11.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
city_list = city_wx_df['City'].tolist()
humidity_rate = city_wx_df['humidity'].astype(float)

marker_locations = city_wx_df[['lat', 'lon']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                            info_box_content=[f'Ideal {city}' for city in city_list])
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity_rate,
                               dissipating=False, max_intensity=100,
                               point_radius=1)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find ideal weather condition
# Max temperature lower than 80 degrees but higher than 70
ideal_df = city_wx_df[city_wx_df['max temperature'] < 80]
ideal_df = ideal_df[ideal_df['max temperature'] > 70]

# Wind speed less than 10 mph
ideal_df = ideal_df[ideal_df['windspeed'] < 10]

# Zero cloudiness
ideal_df = ideal_df[ideal_df['clouds'] == 0]

ideal_df.dropna()

ideal_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
1,Abu Dhabi,AE,292968,54.3667,24.4667,70.83,71.60,78,0,5.75
63,Busselton,AU,2075265,115.3333,-33.6500,70.75,70.75,63,0,6.42
68,Diapaga,BF,2361373,1.7889,12.0708,72.27,72.27,18,0,7.23
119,Saint-Philippe,RE,935215,55.7679,-21.3585,73.22,77.00,69,0,6.91
151,Gayéri,BF,2360541,0.4931,12.6497,71.31,71.31,12,0,8.50
212,Ténenkou,ML,2449938,-4.9169,14.4572,72.07,72.07,10,0,8.30
239,Saint-Joseph,RE,6690296,55.6167,-21.3667,75.09,75.20,73,0,6.91
269,Betioky,MG,1066310,44.3833,-23.7167,75.69,75.69,83,0,8.12
354,Rio Grande,BR,3451138,-52.0986,-32.0350,73.99,73.99,97,0,9.57
401,Koutiala,ML,2454268,-5.4642,12.3917,74.32,74.32,17,0,9.24


In [6]:
ideal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 1 to 547
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             15 non-null     object 
 1   Country          15 non-null     object 
 2   City ID          15 non-null     int64  
 3   lon              15 non-null     float64
 4   lat              15 non-null     float64
 5   temperature      15 non-null     float64
 6   max temperature  15 non-null     float64
 7   humidity         15 non-null     int64  
 8   clouds           15 non-null     int64  
 9   windspeed        15 non-null     float64
dtypes: float64(5), int64(3), object(2)
memory usage: 1.3+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_df[['City','Country','lat','lon']]
hotel_df['Hotel Name'] = ""
hotel_df['Rating'] = ""
hotel_df.reset_index(level=None,drop=True,inplace=True)
hotel_df
 

C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,lat,lon,Hotel Name,Rating
0,Abu Dhabi,AE,24.4667,54.3667,,
1,Busselton,AU,-33.6500,115.3333,,
2,Diapaga,BF,12.0708,1.7889,,
3,Saint-Philippe,RE,-21.3585,55.7679,,
4,Gayéri,BF,12.6497,0.4931,,
5,Ténenkou,ML,14.4572,-4.9169,,
6,Saint-Joseph,RE,-21.3667,55.6167,,
7,Betioky,MG,-23.7167,44.3833,,
8,Rio Grande,BR,-32.0350,-52.0986,,
9,Koutiala,ML,12.3917,-5.4642,,


In [12]:
#
#coords = "-33.9180,25.5701"
target_type = 'lodging'
radius = 5000
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#city_list = ['Port Elizabeth']   #, 'New Norfolk']
#coords = '-33.9180,25.5701'    #,-42.7826	147.0587]

nearest_hotels = []
location_list = []
hotel_ratings = []
heat_map_city_list = []
heat_map_locations = []




for index, row in  hotel_df.iterrows():
    print(row['City'])
    print('*********************************************************')
    #row = hotel_df.loc[hotel_df['City'] == city]
    ##print(row)
    
    latitude = row['lat']
    longitude = row['lon']
    
    coords = f"{latitude},{longitude}"
    location_list.append(coords)
    
    #print (coords)
    params = {
        'location':coords,
        'types': target_type,
        'radius': radius,
        'key': g_key
    }
    
    hotels = requests.get(base_url, params).json()
    
    #print(json.dumps(hotels, indent=4, sort_keys=True))
    
    try:
        hotel_name = hotels['results'][0]['name']
        #hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
        
        rating = hotels['results'][0]['rating']
        #hotel_df.loc[index,'Rating'] = hotels['results'][0]['rating']
        hotel_df.loc[index,'Rating'] = rating
    except:
        fred = row['City']
        print(f'EXCEPTION:{fred} is missing ?')
 
    
hotel_df

Abu Dhabi
*********************************************************
Busselton
*********************************************************
Diapaga
*********************************************************
EXCEPTION:Diapaga is missing ?
Saint-Philippe
*********************************************************
Gayéri
*********************************************************
EXCEPTION:Gayéri is missing ?
Ténenkou
*********************************************************
EXCEPTION:Ténenkou is missing ?
Saint-Joseph
*********************************************************
Betioky
*********************************************************
Rio Grande
*********************************************************
Koutiala
*********************************************************
Igbo-Ora
*********************************************************
EXCEPTION:Igbo-Ora is missing ?
Oussouye
*********************************************************
Río Cuarto
**************************************************

,City,Country,lat,lon,Hotel Name,Rating
0,Abu Dhabi,AE,24.4667,54.3667,Ramee Garden Hotel Apartments,3.6
1,Busselton,AU,-33.6500,115.3333,Observatory Guest House,4.5
2,Diapaga,BF,12.0708,1.7889,,
3,Saint-Philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité""",3.3
4,Gayéri,BF,12.6497,0.4931,,
5,Ténenkou,ML,14.4572,-4.9169,,
6,Saint-Joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast",4.8
7,Betioky,MG,-23.7167,44.3833,Bungalows Acacias Hotel,4.8
8,Rio Grande,BR,-32.0350,-52.0986,Hotel Atlântico Rio Grande,4.3
9,Koutiala,ML,12.3917,-5.4642,Hotel Campement Poule Verte,3.6


In [13]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        15 non-null     object 
 1   Country     15 non-null     object 
 2   lat         15 non-null     float64
 3   lon         15 non-null     float64
 4   Hotel Name  15 non-null     object 
 5   Rating      15 non-null     object 
dtypes: float64(2), object(4)
memory usage: 848.0+ bytes


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# I can't get this line of code to work, so I am going back to what worked for me earlier
locations = hotel_df[["lat", "lon"]]




In [17]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
hotel_markers = gmaps.marker_layer(locations,hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights=None,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(hotel_markers)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))